## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps


# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Excel\WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,0,Victoria,HK,70.00,scattered clouds,22.2855,114.1577,Mini Hotel Central
1,1,Hermanus,ZA,80.01,few clouds,-34.4187,19.2345,Misty Waves Boutique Hotel
2,2,Bengkulu,ID,80.64,overcast clouds,-3.8004,102.2655,Grage Hotel Bengkulu
3,3,Swan Hill,AU,77.02,clear sky,-35.3378,143.5544,Lazy River Motor Inn
4,4,Pisco,PE,68.00,clear sky,-13.7000,-76.2167,Hotel San Isidro Oficial


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 25
What is the maximum temperature you would like for your trip? 96


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,0,Victoria,HK,70.00,scattered clouds,22.2855,114.1577,Mini Hotel Central
1,1,Hermanus,ZA,80.01,few clouds,-34.4187,19.2345,Misty Waves Boutique Hotel
2,2,Bengkulu,ID,80.64,overcast clouds,-3.8004,102.2655,Grage Hotel Bengkulu
3,3,Swan Hill,AU,77.02,clear sky,-35.3378,143.5544,Lazy River Motor Inn
4,4,Pisco,PE,68.00,clear sky,-13.7000,-76.2167,Hotel San Isidro Oficial
5,5,Castro,BR,64.99,overcast clouds,-24.7911,-50.0119,CHACARA BAILLY
6,6,Kruisfontein,ZA,82.99,clear sky,-34.0033,24.7314,Oyster Bay House Rental
7,7,Bluff,NZ,55.80,moderate rain,-46.6000,168.3333,Bluff Homestead - Guesthouse & Campervan Park
8,8,Hithadhoo,MV,82.67,broken clouds,-0.6000,73.0833,Scoop Guest House
9,9,Avarua,CK,75.20,few clouds,-21.2078,-159.7750,Paradise Inn


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                360
City                   360
Country                360
Max Temp               360
Current Description    360
Lat                    360
Lng                    360
Hotel Name             360
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)].dropna()
clean_df.count()

City_ID                360
City                   360
Country                360
Max Temp               360
Current Description    360
Lat                    360
Lng                    360
Hotel Name             360
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
clean_hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
clean_hotel_df["Hotel Name"] = ""
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Victoria,HK,70.00,scattered clouds,22.2855,114.1577,
1,Hermanus,ZA,80.01,few clouds,-34.4187,19.2345,
2,Bengkulu,ID,80.64,overcast clouds,-3.8004,102.2655,
3,Swan Hill,AU,77.02,clear sky,-35.3378,143.5544,
4,Pisco,PE,68.00,clear sky,-13.7000,-76.2167,
5,Castro,BR,64.99,overcast clouds,-24.7911,-50.0119,
6,Kruisfontein,ZA,82.99,clear sky,-34.0033,24.7314,
7,Bluff,NZ,55.80,moderate rain,-46.6000,168.3333,
8,Hithadhoo,MV,82.67,broken clouds,-0.6000,73.0833,
9,Avarua,CK,75.20,few clouds,-21.2078,-159.7750,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key 
}

# 6b. Iterate through the hotel DataFrame.
for index, row in clean_hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        clean_hotel_df.loc[index, "Hotel Name"] = hotels['results'][0]['name']
    except:
        print("Hotel not found... skipping.")
        continue
        
        

In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Victoria,HK,70.00,scattered clouds,22.2855,114.1577,Mini Hotel Central
1,Hermanus,ZA,80.01,few clouds,-34.4187,19.2345,Misty Waves Boutique Hotel
2,Bengkulu,ID,80.64,overcast clouds,-3.8004,102.2655,Grage Hotel Bengkulu
3,Swan Hill,AU,77.02,clear sky,-35.3378,143.5544,Lazy River Motor Inn
4,Pisco,PE,68.00,clear sky,-13.7000,-76.2167,Hotel San Isidro Oficial
...,...,...,...,...,...,...,...
355,Mizan Teferi,ET,86.72,broken clouds,6.9833,35.5833,Hotel Salayish | ሆቴል ሳላይሽ
356,Wad Madani,SD,88.03,clear sky,14.4012,33.5199,فندق يوغسلافيا
357,Nizwa,OM,84.63,clear sky,22.9333,57.5333,Orient Apartments
358,Sao Jose Da Coroa Grande,BR,79.74,overcast clouds,-8.8978,-35.1478,Crôa Mares Hotel


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Excel\WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))